This model was a preliminary attempt at tackling the Lending Club Dataset. It was, in a word, unsuccessful. We encoutnered numerous challenges in the process, the first of which was the vast bulk of data supplied. Furthermore, the dataset had to be comrpehensviely altered to fit our particular purpose. For instance, numerous columns had to be dropped since they would be unknown at the point of loan approval. The vast swaths of missing values in numerous key features and the bulk of object type features also made the task more challenging. Model 3 is better at modelling this dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Dataset 2.gzip', low_memory = False)

In [3]:
df.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,1077501,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1,1077430,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,2,1077175,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,3,1076863,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,4,1075358,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925493 entries, 0 to 2925492
Columns: 142 entries, Unnamed: 0 to debt_settlement_flag
dtypes: float64(106), int64(1), object(35)
memory usage: 3.1+ GB


## Creating A Target Feature

In [5]:
df['loan_status']

0           Fully Paid
1          Charged Off
2           Fully Paid
3           Fully Paid
4           Fully Paid
              ...     
2925488    Charged Off
2925489     Fully Paid
2925490    Charged Off
2925491     Fully Paid
2925492    Charged Off
Name: loan_status, Length: 2925493, dtype: object

In [6]:
df['loan_status'].value_counts()

Fully Paid                                             1497783
Current                                                1031016
Charged Off                                             362548
Late (31-120 days)                                       16154
In Grace Period                                          10028
Late (16-30 days)                                         2719
Issued                                                    2062
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                    433
Name: loan_status, dtype: int64

In [7]:
# compilation of indices where the type of loan status is not needed
unwanted_indices = df[(df['loan_status'] == 'Current') | (df['loan_status'] == 'Issued') | (df['loan_status'] == 'Does not meet the credit policy. Status:Fully Paid') | (df['loan_status'] == 'Does not meet the credit policy. Status:Charged Off') | (df['loan_status'] == 'Default')].index

In [8]:
# dropping data where the type of loan status is not required
df.drop(index = unwanted_indices, inplace = True)

In [9]:
df['loan_status'].value_counts()

Fully Paid            1497783
Charged Off            362548
Late (31-120 days)      16154
In Grace Period         10028
Late (16-30 days)        2719
Name: loan_status, dtype: int64

In [10]:
df['loan_repaid'] = df['loan_status'].apply(lambda status: 1 if status == 'Fully Paid' else 0)

In [11]:
df['loan_repaid']

0          1
1          0
2          1
3          1
4          1
          ..
2925488    0
2925489    1
2925490    0
2925491    1
2925492    0
Name: loan_repaid, Length: 1889233, dtype: int64

In [12]:
# Loan Repaid Is Target Col
df['loan_repaid'].value_counts()

1    1497783
0     391450
Name: loan_repaid, dtype: int64

In [13]:
df.drop('loan_status', axis = 1, inplace = True)

## Feature Engineering

In [14]:
list(df.columns)

['Unnamed: 0',
 'id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'pymnt_plan',
 'url',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verificati

In [15]:
df.drop(['Unnamed: 0', 'id'], axis = 1, inplace = True)

In [16]:
len(df.columns)

140

In [17]:
# delete columns with a high number of null values
for col in df.columns:
    if df[col].isnull().sum() > len(df)/3:
        df.drop(col, axis = 1, inplace = True)

In [18]:
for col in df.columns:
    if df[col].isnull().sum() > 1:
        print(col, ':', df[col].isnull().sum())

emp_title : 135905
emp_length : 120254
title : 21502
zip_code : 2
dti : 1162
inq_last_6mths : 2
revol_util : 1444
last_pymnt_d : 3365
last_credit_pull_d : 66
collections_12_mths_ex_med : 57
tot_coll_amt : 67528
tot_cur_bal : 67528
total_rev_hi_lim : 67528
acc_open_past_24mths : 47282
avg_cur_bal : 67581
bc_open_to_buy : 68157
bc_util : 69184
chargeoff_within_12_mths : 57
mo_sin_old_il_acct : 122304
mo_sin_old_rev_tl_op : 67529
mo_sin_rcnt_rev_tl_op : 67529
mo_sin_rcnt_tl : 67528
mort_acc : 47282
mths_since_recent_bc : 66837
mths_since_recent_inq : 234416
num_accts_ever_120_pd : 67528
num_actv_bc_tl : 67528
num_actv_rev_tl : 67528
num_bc_sats : 55842
num_bc_tl : 67528
num_il_tl : 67528
num_op_rev_tl : 67528
num_rev_accts : 67529
num_rev_tl_bal_gt_0 : 67528
num_sats : 55842
num_tl_120dpd_2m : 136665
num_tl_30dpd : 67528
num_tl_90g_dpd_24m : 67528
num_tl_op_past_12m : 67528
pct_tl_nvr_dlq : 67682
percent_bc_gt_75 : 68597
pub_rec_bankruptcies : 698
tax_liens : 40
tot_hi_cred_lim : 67528
to

In [19]:
df.drop(['emp_title', 'zip_code', 'inq_last_6mths', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'total_rev_hi_lim', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'hardship_flag', 'mort_acc', 'acc_open_past_24mths', ], axis = 1, inplace = True)

In [20]:
for col in df.columns:
    if df[col].isnull().sum() > 1:
        print(col, ':', df[col].isnull().sum())

emp_length : 120254
title : 21502
dti : 1162
tot_coll_amt : 67528
tot_cur_bal : 67528
chargeoff_within_12_mths : 57
num_accts_ever_120_pd : 67528
num_sats : 55842
num_tl_op_past_12m : 67528
pct_tl_nvr_dlq : 67682
pub_rec_bankruptcies : 698
tax_liens : 40


In [21]:
df['emp_length'].value_counts()

10+ years    619963
2 years      170939
< 1 year     157069
3 years      151481
1 year       124803
5 years      117606
4 years      113968
6 years       86561
8 years       79556
7 years       79093
9 years       67940
Name: emp_length, dtype: int64

In [22]:
df['emp_length'].fillna('4 years', inplace = True)
df['emp_length'] = df['emp_length'].apply(lambda x: '1 year' if str(x)[0] == '<' else x)
df['emp_length'] = df['emp_length'].apply(lambda x: 1 if int(str(x)[:2]) > 5 else 0)

In [23]:
df['emp_length']

0          1
1          0
2          1
3          1
4          0
          ..
2925488    0
2925489    1
2925490    1
2925491    0
2925492    0
Name: emp_length, Length: 1889233, dtype: int64

In [24]:
df['title'].value_counts().head(5)

Debt consolidation         957796
Credit card refinancing    372785
Home improvement           112949
Other                      104137
Major purchase              36909
Name: title, dtype: int64

In [25]:
df['title'].fillna('Other', inplace = True)
df['title'] = df['title'].apply(lambda x: 'Other' if x != 'Debt consolidation' and x != 'Credit card refinancing' and x != 'Home improvement' else x)

In [26]:
for col in df.columns:
    if df[col].isnull().sum() > 0:
        print(col, ':', df[col].isnull().sum())

loan_amnt : 1
funded_amnt : 1
funded_amnt_inv : 1
term : 1
int_rate : 1
installment : 1
grade : 1
sub_grade : 1
home_ownership : 1
annual_inc : 1
verification_status : 1
issue_d : 1
pymnt_plan : 1
url : 1
purpose : 1
addr_state : 1
dti : 1162
delinq_2yrs : 1
earliest_cr_line : 1
fico_range_low : 1
fico_range_high : 1
open_acc : 1
pub_rec : 1
revol_bal : 1
total_acc : 1
initial_list_status : 1
out_prncp : 1
out_prncp_inv : 1
total_pymnt : 1
total_pymnt_inv : 1
total_rec_prncp : 1
total_rec_int : 1
total_rec_late_fee : 1
recoveries : 1
collection_recovery_fee : 1
last_pymnt_amnt : 1
last_fico_range_high : 1
last_fico_range_low : 1
policy_code : 1
application_type : 1
acc_now_delinq : 1
tot_coll_amt : 67528
tot_cur_bal : 67528
chargeoff_within_12_mths : 57
delinq_amnt : 1
num_accts_ever_120_pd : 67528
num_sats : 55842
num_tl_op_past_12m : 67528
pct_tl_nvr_dlq : 67682
pub_rec_bankruptcies : 698
tax_liens : 40
debt_settlement_flag : 1


In [27]:
df.dropna(inplace = True) # this doesn't drop too many rows since most of the null values are concentrated in a few rows

In [28]:
df.reset_index(inplace = True) # index is scattered because null values have been dropped - this resets index
df.drop('index', axis = 1, inplace = True) # drops new index col

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820390 entries, 0 to 1820389
Data columns (total 55 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   loan_amnt                 float64
 1   funded_amnt               float64
 2   funded_amnt_inv           float64
 3   term                      object 
 4   int_rate                  object 
 5   installment               float64
 6   grade                     object 
 7   sub_grade                 object 
 8   emp_length                int64  
 9   home_ownership            object 
 10  annual_inc                float64
 11  verification_status       object 
 12  issue_d                   object 
 13  pymnt_plan                object 
 14  url                       object 
 15  purpose                   object 
 16  title                     object 
 17  addr_state                object 
 18  dti                       float64
 19  delinq_2yrs               float64
 20  earliest_cr_line        

In [30]:
df.drop(['funded_amnt', 'funded_amnt_inv', 'last_pymnt_amnt', 'tot_coll_amt', 'out_prncp', 'loan_amnt', 'int_rate', 'installment',  'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'collection_recovery_fee', 'last_fico_range_high', 'last_fico_range_low', 'policy_code', 'pub_rec_bankruptcies', 'tax_liens', 'pct_tl_nvr_dlq', 'recoveries', 'num_accts_ever_120_pd', 'num_tl_op_past_12m', 'chargeoff_within_12_mths', 'revol_bal'], axis = 1, inplace = True)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820390 entries, 0 to 1820389
Data columns (total 29 columns):
 #   Column                Dtype  
---  ------                -----  
 0   term                  object 
 1   grade                 object 
 2   sub_grade             object 
 3   emp_length            int64  
 4   home_ownership        object 
 5   annual_inc            float64
 6   verification_status   object 
 7   issue_d               object 
 8   pymnt_plan            object 
 9   url                   object 
 10  purpose               object 
 11  title                 object 
 12  addr_state            object 
 13  dti                   float64
 14  delinq_2yrs           float64
 15  earliest_cr_line      object 
 16  fico_range_low        float64
 17  fico_range_high       float64
 18  open_acc              float64
 19  pub_rec               float64
 20  total_acc             float64
 21  initial_list_status   object 
 22  application_type      object 
 23  acc_now

In [32]:
# dropping object cols
for x in df.columns:
    if df[x].dtype == object:
        df.drop(x, axis = 1, inplace = True)

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
lst = list(df.columns)

In [35]:
df.columns

Index(['emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low',
       'fico_range_high', 'open_acc', 'pub_rec', 'total_acc', 'acc_now_delinq',
       'tot_cur_bal', 'delinq_amnt', 'num_sats', 'loan_repaid'],
      dtype='object')

In [36]:
lst.remove('loan_repaid')

In [45]:
lst.append('loan_repaid')

In [46]:
X = df.loc[0:10000][lst]
y = df.loc[0:10000]['loan_repaid']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # split the data into train and test sets

In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
model = LogisticRegression(max_iter = 10000) # instantiate the model

In [50]:
model.fit(X_train, y_train) # train the model

LogisticRegression(max_iter=10000)

In [51]:
predictions = model.predict(X_test) # make predictions

In [52]:
from sklearn.metrics import confusion_matrix, classification_report

In [54]:
print(confusion_matrix(y_test, predictions))
print()
print(classification_report(y_test, predictions))

[[   0  477]
 [   0 2524]]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       477
           1       0.84      1.00      0.91      2524

    accuracy                           0.84      3001
   macro avg       0.42      0.50      0.46      3001
weighted avg       0.71      0.84      0.77      3001



C:\Users\sirki\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sirki\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sirki\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
